In [4]:
import pandas as pd
# 과학적 표기 안하도록
pd.options.display.float_format = '{:.2f}'.format

import numpy as np
import datetime

---
### request+bs로 크롤링


In [7]:
import requests
from bs4 import BeautifulSoup as bs

In [8]:
# code로 접근 가능한 url
code_url = "https://finance.naver.com/item/main.nhn?code="
# 종목 분석 url
coinfo_url = "https://finance.naver.com/item/coinfo.nhn?code="
# 종목 분석 url (!naver)
coinfo_url_2 = "https://navercomp.wisereport.co.kr/v2/company/c1050001.aspx?cmp_cd=%s&cn="
# 시가총액 목록 url
size_market_url_0 = "https://finance.naver.com/sise/sise_market_sum.nhn?sosok=0&page="
size_market_url_1 = "https://finance.naver.com/sise/sise_market_sum.nhn?sosok=1&page="

---

### 시가총액 목록 사용하기

In [ ]:
# 오늘의 시가총액 상위 400 code 목록 구하기

codeL = []
# 코스피 상위 300
for i in range(1,7):
    soup = bs(requests.get(size_market_url_0+str(i)).text, 'html.parser')
    tempL = [x.attrs["href"].split("code=")[1] for x in soup.select("table ")[1].select_one("tbody").select("a.tltle")]
    codeL += tempL
# 코스닥 상위 100
for i in range(1, 3):
    soup = bs(requests.get(size_market_url_1+str(i)).text, 'html.parser')
    tempL = [x.attrs["href"].split("code=")[1] for x in soup.select("table ")[1].select_one("tbody").select("a.tltle")]
    codeL += tempL

In [7]:
# 오늘의 시가총액 code 목록 저장 
todayDate = datetime.datetime.now().strftime("%Y_%b_%d")
f = open("./"+todayDate+"_시가총액_상위권.txt", 'w')
f.write("\n".join(codeL))
f.close()

# 오늘의 시가총액 code 목록 불러오기
useDate = "2020_Feb_18"
f = open("./"+useDate+"_시가총액_상위권.txt", "r")
codeL = [x.replace("\n", "") for x in f.readlines()]
f.close()

----

##### 테마별 업종 목록 사용하기

In [8]:
# 테마
# theme_no = 14
# theme_url = "https://finance.naver.com/sise/sise_group_detail.nhn?type=theme&no="
# soup = bs(requests.get(theme_url + str(theme_no)).text, 'html.parser')
# codeL = [x.select_one("a").attrs["href"].split("code=")[1] for x in soup.select("table")[2].select("div.name_area")]

---

### 컨센서스 확인 함수

In [9]:
def getName(code):
    code_url = "https://finance.naver.com/item/main.nhn?code="
    soup = bs(requests.get(code_url + code).text, 'html.parser')
    name = soup.select_one("#middle > div.h_company > div.wrap_company > h2 > a").text
    return name

In [10]:
def getPER(code):
    code_url = "https://finance.naver.com/item/main.nhn?code="
    soup = bs(requests.get(code_url + code).text, 'html.parser')
    try:
        myPer = soup.select_one("table.per_table").select_one("td").select_one("em").text
        avgPer = soup.select("tr.strong > td > em")[-1].text
    except:
        myPer = 0
        avgPer = 0
        return myPer, avgPer
    if(myPer == "N/A"):
        myPer = 0
    if(avgPer == "N/A"):
        avgPer = 0
    return myPer, avgPer

In [11]:
def Consensus(code, score25, score23, score20):
    code = str(code)
    code_url = "https://finance.naver.com/item/main.nhn?code="
        
    # 종목 이름 얻기
    name = getName(code)
    
    # 컨센서스 테이블 구축
    consenUrl = "https://navercomp.wisereport.co.kr/company/ajax/c1050001_data.aspx?flag=2&cmp_cd="\
                + code + "&finGubun=MAIN&frq=0&chartType=svg"
    
    # 컨센서스 없는 경우 1
    if(requests.get(consenUrl).json()["JsonData"] == []):
        return 
    df_table = pd.DataFrame(requests.get(consenUrl).json()["JsonData"]).iloc[:, :-1]

    # 매출액 ~ EV/EBITDA 숫자형으로 변경
    df_table.iloc[:, 1:-1] = df_table.iloc[:,1:-1].applymap(lambda x: "0" if x == "" else x).applymap(lambda x: float(x.replace(",","")))
    
    # 컨센서스 없는 경우 2
    if(df_table.iloc[:, 1:-1].sum().sum() == 0):
        return
    
    # 증감 결과 df table로 확인
    result_row = [0]*12
    for inc in [1, 2, 3, 4, 5, 6, 9]:
        if(increasing(df_table.iloc[:, inc])):
            result_row[inc] = 1
    for dec in [7, 8, 10]:
        if(decreasing(df_table.iloc[:, dec])):
            result_row[dec] = 1

#     result_df = pd.DataFrame([result_row], columns = df_table.columns)
#     df_table.append(result_df, ignore_index = True)
    
    # 증감 결과 매출액 / 영업이익 / per * 10 roe * 5
    formula = [0,10,1,10,1,1,1,10,1,5,1,0]
    score = np.dot(result_row,formula)

    # 점수 결과
    return [code, name, str(score)]


---

## 시가총액 상위 400개의 컨센서스 확인

In [12]:
score25 = []
score23 = [] 
score20 = []
for i in range(len(codeL)):
    if(i % 100 == 99):
        print(i)
        
    # per 확인
    myPer, avgPer = getPER(codeL[i])
    if(myPer == 0 or avgPer == 0):
        continue
    if(float(myPer.replace(",","")) > float(avgPer.replace(",",""))):
        continue
    
    result = Consensus(codeL[i], score25, score23, score20)
    if(result):
        if(int(result[2]) >= 25):
            score25.append(" ".join(result))
        elif(int(result[2]) >= 23):
            score23.append(" ".join(result))
        elif(int(result[2]) >= 20):
            score20.append(" ".join(result))
    

99
199
299
399


In [13]:
score25

['033290 코웰패션 33']

In [14]:
score23

['051900 LG생활건강 23',
 '012510 더존비즈온 23',
 '003230 삼양식품 23',
 '030190 NICE평가정보 23',
 '035900 JYP Ent. 23',
 '067160 아프리카TV 23',
 '267980 매일유업 23',
 '215200 메가스터디교육 23',
 '272290 이녹스첨단소재 23']

In [15]:
score20

['004170 신세계 22',
 '111770 영원무역 20',
 '192080 더블유게임즈 21',
 '286940 롯데정보통신 21',
 '145720 덴티움 21',
 '253450 스튜디오드래곤 21',
 '036830 솔브레인 21']

---

#### 오늘의 점수 저장

In [63]:
score25_code = [x.split()[0] for x in score25]
score23_code = [x.split()[0] for x in score23]
score20_code = [x.split()[0] for x in score20]

In [64]:
# 오늘의 socre 저장 
todayDate = datetime.datetime.now().strftime("%Y_%b_%d")
f = open("./scoreData/"+todayDate+"_SCORE>=25.txt", 'w')
f.write("\n".join(score25_code))
f.close()

f = open("./scoreData/"+todayDate+"_SCORE>=23.txt", 'w')
f.write("\n".join(score23_code))
f.close()

f = open("./scoreData/"+todayDate+"_SCORE>=20.txt", 'w')
f.write("\n".join(score20_code))
f.close()

In [12]:
todayDate = datetime.datetime.now().strftime("%Y_%b_%d")


In [13]:
todayDate


'2020_Feb_19'

In [12]:
# 오늘의 socre 불러오기
todayDate = datetime.datetime.now().strftime("%Y_%b_%d")
todayDate = "2020_Feb_18"

f = open("./scoreData/"+todayDate+"_SCORE>=25.txt", 'r')
score25_code = [x.replace("\n", "") for x in f.readlines()]
f.close()

f = open("./scoreData/"+todayDate+"_SCORE>=23.txt", 'r')
score23_code = [x.replace("\n", "") for x in f.readlines()]
f.close()

f = open("./scoreData/"+todayDate+"_SCORE>=20.txt", 'r')
score20_code = [x.replace("\n", "") for x in f.readlines()]
f.close()

---

## 나의 네이버증권 셋팅하기

In [13]:
from selenium import webdriver
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.common.keys import Keys

import time

In [14]:
# 미리 로그인
driver = webdriver.Chrome('./chromedriver')
itemListUrl = "https://finance.naver.com/mystock/itemList.nhn?groupId=6&type=OVERALL"
driver.get(itemListUrl)

In [16]:
for group in range(6, 9):

    # 링크 접속 및 로그인
    itemListUrl = "https://finance.naver.com/mystock/itemList.nhn?groupId=%s&type=OVERALL" % str(group)
    print(itemListUrl )
    
    # 기존 관심 종목 삭제
    time.sleep(0.5)
    driver.get(itemListUrl)
    time.sleep(0.5)
    driver.find_element_by_css_selector("#all_select").click()
    time.sleep(0.5)
    driver.find_element_by_css_selector("#wrap > div.section_mys > div.group_mystb.NE\=a\:lst > div:nth-child(2) > button.btn_del.NPI\=a\:del").click()
    time.sleep(0.5)
    Alert(driver).accept()
    
    # 적합 종목 리스트 찾기
    if(group == 6):
        useL = score25_code.copy()
    elif(group == 7):
        useL = score23_code.copy()
    elif(group == 8):
        useL = score20_code.copy()
            
    # 관심종목 목록 등록
    for nowCode in useL:
        time.sleep(0.5)
        driver.find_element_by_css_selector("#mystock_input").clear()
        time.sleep(0.5)
        driver.find_element_by_css_selector("#mystock_input").send_keys(nowCode)
        driver.find_element_by_css_selector("#mystock_input").send_keys(Keys.RETURN)


https://finance.naver.com/mystock/itemList.nhn?groupId=6&type=OVERALL
https://finance.naver.com/mystock/itemList.nhn?groupId=7&type=OVERALL
https://finance.naver.com/mystock/itemList.nhn?groupId=8&type=OVERALL
